# Modelado
Aquí modelaremos

In [27]:
# dependencies
import pandas as pd
from   sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
# data
path = 'work/data/processed/celulares_procesasdos.csv'

In [3]:
# loading
df_inicio = pd.read_csv(path)

In [4]:
# conservo columnas a usar
columnas_ignorar = {'color', 'pantalla'}
df = df_inicio.drop(columns = columnas_ignorar).drop_duplicates().reset_index(drop = True)

In [5]:
df

,marca,producto_nombre,peso,camara_trasera,camara_frontal,procesador,ram,memoria,sistema_operativo,precio,tecnologia
0,samsung,galaxy z fold2,0.282,12,10,qualcomm,12,256,android,46799,5g
1,apple,iphone 11 pro max,0.272,12,12,apple,4,512,ios,33999,4glte
2,apple,iphone 11 pro,0.252,12,12,apple,4,512,ios,31279,4glte
3,samsung,galaxy note 20 ultra,0.302,12,10,qualcomm,8,256,android,30599,4g
4,samsung,galaxy z flip,0.183,12,10,qualcomm,8,256,android,29699,4glte
...,...,...,...,...,...,...,...,...,...,...,...
76,motorola,moto e6 plus,0.149,13,8,qualcomm,2,32,android,2990,4glte
77,motorola,moto e6 play,0.190,13,8,qualcomm,1,16,android,2499,4glte
78,motorola,moto e6 plus,0.149,13,8,mediatek,2,32,android,2990,4glte
79,huawei,honor 8a,0.176,13,8,mediatek,2,32,android,3299,4glte


In [7]:
# columnas categoricas
columnas_categoricas = ['marca', 'procesador', 'sistema_operativo', 'tecnologia']
# columnas numericas
columnas_numericas = ['peso', 'camara_trasera', 'camara_frontal', 'ram', 'memoria', 'precio']

In [29]:
# normalización de variables continuas
estandarizador = StandardScaler()
estandarizador.fit(df[columnas_numericas])
estandarizador.transform(df[columnas_numericas])

array([[ 0.98322039, -0.58174055, -0.54273998,  3.38558314,  0.93377809,
         3.58893158],
       [ 0.80725739, -0.58174055, -0.33239578, -0.39042009,  3.10644668,
         2.22155124],
       [ 0.4553314 , -0.58174055, -0.33239578, -0.39042009,  3.10644668,
         1.93098292],
       [ 1.33514638, -0.58174055, -0.54273998,  1.49758153,  0.93377809,
         1.85834084],
       [-0.75881326, -0.58174055, -0.54273998,  1.49758153,  0.93377809,
         1.76219691],
       [ 0.80725739, -0.58174055, -0.33239578, -0.39042009,  0.93377809,
         1.68645685],
       [ 1.51110937, -0.58174055, -0.54273998,  1.49758153,  0.93377809,
         1.36597709],
       [ 0.4553314 , -0.58174055, -0.33239578, -0.39042009,  0.93377809,
         1.27720419],
       [ 0.4553314 , -0.58174055, -0.33239578, -0.39042009,  0.93377809,
         1.2591505 ],
       [ 0.4553314 , -0.58174055, -0.33239578, -0.39042009,  0.93377809,
         1.2057372 ],
       [ 1.44072417, -0.58174055, -0.85825629, -0.

In [28]:
# codificación de variables categoricas
# one hot encoder
encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
encoder.fit(df[columnas_categoricas]).transform(df[columnas_categoricas])

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.]])

In [12]:
df[columnas_categoricas]

,marca,procesador,sistema_operativo,tecnologia
0,samsung,qualcomm,android,5g
1,apple,apple,ios,4glte
2,apple,apple,ios,4glte
3,samsung,qualcomm,android,4g
4,samsung,qualcomm,android,4glte
...,...,...,...,...
76,motorola,qualcomm,android,4glte
77,motorola,qualcomm,android,4glte
78,motorola,mediatek,android,4glte
79,huawei,mediatek,android,4glte
